In [ ]:
#| default_exp tui


# TUI - Terminal User Interface

Textual-based terminal user interface components for audio transcription.


In [ ]:
#| export
import threading
import time
from pathlib import Path

from textual.app import App, ComposeResult
from textual.containers import Horizontal, Vertical
from textual.screen import ModalScreen
from textual.widgets import Button, DataTable, Footer, Header, Static, TextArea

from tui_writer.audio import AudioRecorder
from tui_writer.transcription import WhisperTranscriber
from tui_writer.utils import copy_to_clipboard, format_duration


## ModelsScreen

Modal dialog for selecting Whisper transcription models.


In [ ]:
#| export

class ModelsScreen(ModalScreen):
    """Modal dialog for selecting Whisper transcription models."""

    def __init__(self, transcriber=None, **kwargs):
        super().__init__(**kwargs)
        self.models = WhisperTranscriber.VALID_MODELS
        self.transcriber = transcriber
        self.selected_row = 0

    def compose(self) -> ComposeResult:
        current_model = self.transcriber.model_name if self.transcriber else "None"
        yield Header()
        yield Vertical(
            Static("🎯 Available Whisper Models", id="models-title", classes="success"),
            Static(f"Current model: [bold cyan]{current_model}[/bold cyan]", id="current-model"),
            Static("Select a model for transcription (use ↑↓ arrows, then click Select):", id="models-subtitle"),
            DataTable(id="models-table", cursor_type="row"),
            Horizontal(
                Button("Select Model", variant="success", id="select-btn"),
                Button("Close", variant="default", id="close-btn"),
                id="models-buttons"
            ),
            id="models-container"
        )
        yield Footer()

    def on_mount(self) -> None:
        """Set up the models table when the screen mounts."""
        table = self.query_one("#models-table", DataTable)

        # Add columns
        table.add_columns("Model Name", "Type", "Description")

        # Add model data
        current_model = self.transcriber.model_name if self.transcriber else None
        for i, model in enumerate(self.models):
            # Categorize models
            if model.endswith(".en"):
                model_type = "English-only"
                description = "Optimized for English speech"
            elif model.startswith("distil-"):
                model_type = "Distilled"
                description = "Faster, smaller version"
            elif "turbo" in model:
                model_type = "Turbo"
                description = "Fastest performance"
            else:
                model_type = "Multilingual"
                description = "Supports multiple languages"

            # Mark current model with indicator
            if model == current_model:
                model = f"▶ {model}"  # Add arrow indicator for current model

            table.add_row(model, model_type, description)

        # Set cursor to current model if found
        if current_model:
            try:
                current_index = self.models.index(current_model)
                table.cursor_cell = (current_index, 0)
            except (ValueError, IndexError):
                table.cursor_cell = (0, 0)

    def on_button_pressed(self, event: Button.Pressed) -> None:
        """Handle button presses in the modal."""
        if event.button.id == "close-btn":
            self.dismiss()
        elif event.button.id == "select-btn":
            self.select_current_model()

    def select_current_model(self) -> None:
        """Select the currently highlighted model and update the transcriber."""
        table = self.query_one("#models-table", DataTable)
        cursor_row, _ = table.cursor_cell

        if 0 <= cursor_row < len(self.models):
            selected_model = self.models[cursor_row]

            # Update the transcriber if available
            if self.transcriber:
                try:
                    # Store the old model name for status message
                    old_model = self.transcriber.model_name
                    
                    # Use the transcriber's update_model method to handle all the logic
                    self.transcriber.update_model(selected_model)

                    # Update the main screen's status to show the new model
                    main_screen = self.app.screen
                    if hasattr(main_screen, 'update_status'):
                        main_screen.update_status(f"✅ Model changed from '{old_model}' to '{selected_model}'")

                    self.notify(f"✅ Model updated: {selected_model}")
                    self.dismiss()
                except Exception as e:
                    self.notify(f"❌ Error updating model: {e}", severity="error")
            else:
                self.notify(f"Selected model: {selected_model}")
                self.dismiss()


## TranscriptionTUI

Main terminal user interface application for audio transcription.

The TUI uses Textual's [theme system](https://textual.textualize.io/guide/design/) with built-in color variables like `$primary`, `$error`, `$success`, etc. 

**Available built-in themes:**
- `textual-dark` (default)
- `textual-light`
- `nord`
- `gruvbox`
- `tokyo-night`
- `solarized-light`

To change the theme, modify the `self.theme` assignment in `__init__`.


In [ ]:
#| export

class TranscriptionTUI(App):
    """Terminal UI for audio transcription with Whisper models."""

    # Use Textual's built-in theme system instead of external CSS
    CSS = """
    Screen {
        layout: vertical;
    }
    
    #status {
        height: 3;
        margin: 1;
    }
    
    #recording-display {
        height: 3;
        margin: 1;
        text-align: center;
    }
    
    #transcription-display {
        height: 1fr;
        margin: 1;
        border: solid $primary;
        padding: 1;
    }
    
    #controls {
        height: 3;
        margin: 1;
    }
    
    Button {
        margin: 0 1;
    }
    
    .recording {
        color: $error;
        text-style: bold;
    }
    
    .transcribing {
        color: $accent;
        text-style: bold;
    }
    
    .success {
        color: $success;
        text-style: bold;
    }
    
    .error {
        color: $error;
        text-style: bold;
    }
    
    /* Modal screen styling */
    ModelsScreen {
        align: center middle;
    }
    
    #models-container {
        width: 80;
        height: 70%;
        border: solid $primary;
        background: $surface;
        padding: 1;
    }
    
    #models-title {
        text-align: center;
        margin-bottom: 1;
    }
    
    #models-subtitle {
        text-align: center;
        margin-bottom: 1;
        color: $text-muted;
    }
    
    #models-table {
        height: 1fr;
    }
    
    #models-buttons {
        margin-top: 1;
        align: center middle;
    }
    """

    BINDINGS = [
        ("r", "start_recording", "Start Recording"),
        ("s", "stop_recording", "Stop Recording"),
        ("t", "transcribe_last", "Transcribe Last Recording"),
        ("l", "list_models", "List Models"),
        ("q", "quit", "Quit"),
        ("c", "copy_to_clipboard", "Copy to Clipboard"),
        ("enter", "stop_and_transcribe", "Stop Recording & Transcribe"),
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.recorder = AudioRecorder()
        self.transcriber = WhisperTranscriber()
        self.recording_timer = None
        self.recording_start_time = None
        self.current_transcription = None
        # Set a theme (you can change this to any built-in theme)
        # Options: "textual-dark", "textual-light", "nord", "gruvbox", "tokyo-night", "solarized-light"
        self.theme = "textual-dark"

    def compose(self) -> ComposeResult:
        """Create the UI layout."""
        yield Header()
        yield Vertical(
            Static("🎤 Audio Transcription TUI", id="title", classes="success"),
            Static("", id="status"),
            Static("", id="recording-display"),
            TextArea("Transcription will appear here...", id="transcription-display", disabled=True),
            Horizontal(
                Button("🎤 Recording... (ENTER to stop & transcribe)", variant="warning", id="recording-status", disabled=True),
                Button("Stop Recording (S)", variant="error", id="stop-btn", disabled=True),
                Button("Transcribe Last (T)", variant="primary", id="transcribe-btn"),
                Button("List Models (L)", variant="default", id="models-btn"),
                Button("Quit (Q)", variant="default", id="quit-btn"),
                id="controls"
            ),
            id="main-container"
        )
        yield Footer()

    def on_mount(self) -> None:
        """Start recording immediately on launch."""
        self.start_recording()

    def on_button_pressed(self, event: Button.Pressed) -> None:
        """Handle button presses."""
        button_id = event.button.id
        if button_id == "start-btn":
            self.start_recording()
        elif button_id == "stop-btn":
            self.stop_recording()
        elif button_id == "recording-status":
            self.action_stop_and_transcribe()
        elif button_id == "transcribe-btn":
            self.transcribe_last()
        elif button_id == "models-btn":
            self.list_models()
        elif button_id == "quit-btn":
            self.action_quit()

    def action_start_recording(self) -> None:
        """Start audio recording."""
        self.start_recording()

    def action_stop_recording(self) -> None:
        """Stop audio recording."""
        self.stop_recording()

    def action_transcribe_last(self) -> None:
        """Transcribe the last recorded audio."""
        self.transcribe_last()

    def action_list_models(self) -> None:
        """List available Whisper models."""
        self.list_models()

    def action_quit(self) -> None:
        """Quit the application."""
        self.exit()

    def action_copy_to_clipboard(self) -> None:
        """Copy current transcription to clipboard."""
        if self.current_transcription:
            copy_to_clipboard(self.current_transcription)
            self.update_status("Transcription copied to clipboard!")
        else:
            self.update_status("No transcription to copy!")

    def action_stop_and_transcribe(self) -> None:
        """Stop recording and transcribe the audio."""
        if self.recorder.recording:
            self.stop_recording()
            self.call_later(self.transcribe_last)

    def start_recording(self) -> None:
        """Start audio recording and update UI."""
        try:
            self.recorder.start_recording()
            self.recording_start_time = time.time()
            self.update_status("🔴 Recording started...")
            self.update_recording_display("Recording... 00:00")
            self.recording_timer = self.set_interval(1.0, self.update_recording_timer)
            self.query_one("#stop-btn", Button).disabled = False
        except Exception as e:
            self.update_status(f"❌ Failed to start recording: {e}", "error")

    def stop_recording(self) -> None:
        """Stop the recording process."""
        if self.recording_timer:
            self.recording_timer.stop()
            self.recording_timer = None
        try:
            self.recorder.stop_recording()
            elapsed = time.time() - self.recording_start_time
            duration_str = format_duration(elapsed)
            self.update_status(f"✅ Recording stopped. Duration: {duration_str}")
            self.update_recording_display(f"Recording saved: {duration_str}")
            self.query_one("#stop-btn", Button).disabled = True
        except Exception as e:
            self.update_status(f"❌ Failed to stop recording: {e}", "error")

    def transcribe_last(self) -> None:
        """Transcribe the last recorded audio file."""
        audio_path = self.recorder.audio_file_path
        if not audio_path.exists():
            self.update_status("❌ No previous recording found. Record audio first.", "error")
            return
        self.update_status("🔄 Transcribing audio...")
        self.query_one("#transcribe-btn", Button).disabled = True

        def do_transcription():
            try:
                transcription, elapsed_time = self.transcriber.transcribe(audio_path)
                self.call_from_thread(self.transcription_complete, transcription, elapsed_time)
            except Exception as e:
                self.call_from_thread(self.update_status, f"❌ Transcription failed: {e}", "error")
                def reset_transcribe_button():
                    self.query_one("#transcribe-btn", Button).update("Transcribe Last (T)")
                    self.query_one("#transcribe-btn", Button).disabled = False
                self.call_from_thread(reset_transcribe_button)
        threading.Thread(target=do_transcription, daemon=True).start()

    def transcription_complete(self, transcription: str, elapsed_time: float) -> None:
        """Called when transcription is complete."""
        self.current_transcription = transcription
        self.query_one("#transcription-display", TextArea).text = transcription
        self.query_one("#transcribe-btn", Button).disabled = False
        copy_to_clipboard(transcription, elapsed_time)
        self.update_status(f"✅ Transcription completed in {elapsed_time:.1f}s")

    def list_models(self) -> None:
        """Open the model selection dialog."""
        self.app.push_screen(ModelsScreen(transcriber=self.transcriber))

    def update_status(self, message: str, style: str = "default") -> None:
        """Update the status display."""
        try:
            status_widget = self.query_one("#status", Static)
            status_widget.update(message)
            if style == "error":
                status_widget.add_class("error")
            elif style == "success":
                status_widget.add_class("success")
            else:
                status_widget.remove_class("error success")
        except Exception:
            pass

    def update_recording_display(self, message: str) -> None:
        """Update the recording display and status button."""
        try:
            display_widget = self.query_one("#recording-display", Static)
            display_widget.update(message)
            status_button = self.query_one("#recording-status", Button)
            if "Recording" in message:
                status_button.label = f"🎤 {message} (ENTER to stop & transcribe)"
                status_button.variant = "warning"
            else:
                status_button.label = "✅ Ready to record"
                status_button.variant = "success"
        except Exception:
            pass

    def update_recording_timer(self) -> None:
        """Update the recording timer display."""
        if self.recording_start_time:
            elapsed = time.time() - self.recording_start_time
            time_str = format_duration(elapsed)
            self.update_recording_display(f"🔴 Recording... {time_str}")
